Based on https://github.com/KevinLTT/video2bvh

**HOW TO**
*  Find out GPU model
*  Install libs/deps
*  Get openpose sources and build
*  Download prebuild or build
*  Try generate BVH


**DO NOT USE LONG VIDEO**

*otherwise it takes forever or bvh could be unusable*


This Virtual Machine will be alive next **12hrs** or until "Runtime->factory reset"

https://research.google.com/colaboratory/faq.html#idle-timeouts



Find out GPU model, is usually one of:

* "Tesla T4"
* "Tesla P4"
* "Tesla P100-PCIE"
* "Tesla K80"

If someting not works - try **Factory reset to obtain another GPU**

In [ ]:
#Get system info
! lsb_release -a
#Get GPU info
! nvcc --version
! nvidia-smi


Download all sources:

In [ ]:
!git init
!echo ".*" > .gitignore
!git remote add origin https://github.com/andriitishchenko/video2bvh.git
!git fetch
!git checkout master
!git submodule update --init --recursive
!cd openpose/models/pose/body_25/ && curl -L -C - http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_25/pose_iter_584000.caffemodel -O

# # download ALL models
# !cd openpose/models/ && ./getModels.sh


* Install Libs

In [ ]:
#Install latest cmake, default is cmake 3.13
! sudo apt-get purge cmake
! curl -L https://github.com/Kitware/CMake/releases/download/v3.18.0-rc3/cmake-3.18.0-rc3-Linux-x86_64.sh -O 
! chmod +x cmake-3.18.0-rc3-Linux-x86_64.sh
! ./cmake-3.18.0-rc3-Linux-x86_64.sh --skip-license --prefix=/usr/local
! cmake --version
! rm cmake-3.18.0-rc3-Linux-x86_64.sh

! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libopencv-dev
! sudo apt-get --assume-yes install build-essential libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev

# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

# OpenCV
! sudo apt-get --assume-yes install libopencv-dev

# Tools
!sudo apt install imagemagick

# ======================
! pip3 install --upgrade pip
! pip3 uninstall albumentations -y
! pip3 install --upgrade opencv-python albumentations
! pip install progressbar2

import sys
sys.path.append('/content/openpose/build/python/openpose')

**Create link to your Google drive for models:**

https://drive.google.com/drive/folders/1M2s32xQkrDhDLz-VqzvocMuoaSGR1MfX

*  Mount and copy models

In [ ]:

from google.colab import drive
import os
# mount Google Drive
# When you mount it, you will be asked for permission, so allow it, 
# copy the key, paste it in the input field on the Colab side, and press Enter.
drive.mount('/gdrive')
# copy pretrained linked models 
!cp -r "/gdrive/My Drive/models/" ./models

Python 3 + cuda **builded**


In [ ]:
### check if GPU one of "Tesla T4","Tesla P4","Tesla P100-PCIE","Tesla K80"
### if GPU not listed - build from sources
### if openpose not works - build from sources
! rm -rf openpose/build/

### Tesla P100-PCIE
# ! curl -L -C - https://github.com/andriitishchenko/openpose_demo/raw/master/ubuntu/build_cuda_py3_Tesla_P100-PCIE.zip -o build.zip

### Tesla P4
# ! curl -L -C - https://github.com/andriitishchenko/openpose_demo/raw/master/ubuntu/build_cuda_py3_Tesla_P4.zip -o build.zip

### Tesla T4
# ! curl -L -C - https://github.com/andriitishchenko/openpose_demo/raw/master/ubuntu/build_cuda_py3_Tesla_T4.zip -o build.zip

### Tesla K80
# ! curl -L -C - https://github.com/andriitishchenko/openpose_demo/raw/master/ubuntu/build_cuda_py3_Tesla_K80.zip -o build.zip
! unzip build.zip -d openpose/
!echo "DONE ..."

###  OR BUILD from sources, it takes ~20min, python3 using
# build_dir="build"
# ! rm -rf "openpose/$build_dir"
# ! mkdir "openpose/$build_dir"
# ! cd "openpose/$build_dir" && cmake .. -DBUILD_PYTHON=On
# ! cd "openpose/$build_dir" && make -j`nproc`
# !echo "DONE ..."



---



In [ ]:
from pose_estimator_2d import openpose_estimator
from pose_estimator_3d import estimator_3d
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import sys
import cv2
import importlib
import numpy as np
from pathlib import Path
from IPython.display import HTML
import time
import progressbar

# set model_folder to /path/to/openpose/models
e2d = openpose_estimator.OpenPoseEstimator(model_folder='openpose/models/') 

# video file to process:
# "miscs/cxk.mp4" - predefined 
video_file = Path("/gdrive/My Drive/bvh/1.mp4") 
output_dir = Path(f'miscs/{video_file.stem}_cache')

!rm -rf $output_dir

if not output_dir.exists():
    os.makedirs(output_dir)
    
cap = cv2.VideoCapture(str(video_file))
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
bar = progressbar.ProgressBar(max_value=totalFrames)
keypoints_list = []
img_width, img_height = None, None
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    nextFrameNo = cap.get(cv2.CAP_PROP_POS_FRAMES)
    bar.update(nextFrameNo)

    img_height = frame.shape[0]
    img_width = frame.shape[1]
    
    # returned shape will be (num_of_human, 25, 3)
    # last dimension includes (x, y, confidence)
    keypoints = e2d.estimate(img_list=[frame])[0]
    if isinstance(keypoints, np.ndarray) and len(keypoints.shape) == 3:
        keypoints_list.append(keypoints[0])

cap.release()
print("")

# filter out failed result
keypoints_list = smooth.filter_missing_value(
    keypoints_list=keypoints_list,
    method='ignore' # interpolation method will be implemented later
)

# smooth process will be implemented later

# save 2d pose result
pose2d = np.stack(keypoints_list)[:, :, :2]
pose2d_file = Path(output_dir / '2d_pose.npy')
np.save(pose2d_file, pose2d)
print("Saved to:")
print(pose2d_file)

In [ ]:
## Visualize 2D pose, not requaried for bvh
## this will fail if video has no people frames

### uncoment if nedded

# cap = cv2.VideoCapture(str(video_file))
# bar = progressbar.ProgressBar(max_value=len(keypoints_list))
# vis_result_dir = output_dir / '2d_pose_vis' # path to save the visualized images
# if not vis_result_dir.exists():
#     os.makedirs(vis_result_dir)
    
# op_skel = openpose_skeleton.OpenPoseSkeleton()

# for i, keypoints in enumerate(keypoints_list):
#     ret, frame = cap.read()
#     if not ret:
#         break
#     ### keypoint whose detect confidence under kp_thresh will not be visualized
#     vis.vis_2d_keypoints(
#         keypoints=keypoints,
#         img=frame,
#         skeleton=op_skel,
#         kp_thresh=0.4,
#         output_file=vis_result_dir / f'{i:04d}.png'
#     )
#     bar.update(i)

# cap.release()
# print("")
# print("Done...")


In [ ]:
# Initialize 3D pose estimator
print("Initialize 3D pose estimator ...")
importlib.reload(estimator_3d)
e3d = estimator_3d.Estimator3D(
    config_file='models/openpose_video_pose_243f/video_pose.yaml',
    checkpoint_file='models/openpose_video_pose_243f/best_58.58.pth'
)

In [ ]:
# Estimate 3D pose from 2D pose
print("Estimate 3D pose from 2D pose ...")
# import torch
# torch.cuda.empty_cache()
pose2d = np.load(pose2d_file)
pose3d = e3d.estimate(pose2d, image_width=img_width, image_height=img_height)


In [ ]:
# convert coordinates
print("convert coordinates ...")
subject = 'S1'
cam_id = '55011271'
cam_params = camera.load_camera_params('cameras.h5')[subject][cam_id]
R = cam_params['R']
T = 0
azimuth = cam_params['azimuth']

pose3d_world = camera.camera2world(pose=pose3d, R=R, T=T)
pose3d_world[:, :, 2] -= np.min(pose3d_world[:, :, 2]) # rebase the height

pose3d_file = output_dir / '3d_pose.npy'
np.save(pose3d_file, pose3d_world)

In [ ]:
### Visualize 3D pose, not requaried for bvh
### uncoment if nedded

# h36m_skel = h36m_skeleton.H36mSkeleton()
# gif_file = f'{output_dir}/3d_pose_300_{video_file.stem}.gif' # output format can be .gif or .mp4 
# print(gif_file)
# ani = vis.vis_3d_keypoints_sequence(
#     keypoints_sequence=pose3d_world[0:300],
#     skeleton=h36m_skel,
#     azimuth=azimuth,
#     fps=60,
#     output_file=gif_file
# )
# HTML(ani.to_jshtml())

In [ ]:
# Create bvh
print("Create bvh ...")
bvh_file = output_dir / f'{video_file.stem}.bvh'
cmu_skel = cmu_skeleton.CMUSkeleton()
channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)

output = f'miscs/h36m_{video_file.stem}.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()
_ = h36m_skel.poses2bvh(pose3d_world, output_file=output)

#perform dowmload
from google.colab import files
files.download(output)

In [ ]:

### Processing mp4 files in Google Drive folder
#           (start time)                   (duration time) 
# ffmpeg -ss 00:01:20 -i input_video.mp4 -t 00:00:10 -vcodec copy -acodec copy -y out_part_1.mp4
#
###

from pose_estimator_2d import openpose_estimator
from pose_estimator_3d import estimator_3d
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import sys
import cv2
import importlib
import numpy as np
from pathlib import Path
from IPython.display import HTML
import time
import progressbar
import os

e2d = openpose_estimator.OpenPoseEstimator(model_folder='openpose/models/') 

directory = r'/gdrive/My Drive/bvh/1080/'
for filename in os.listdir(directory):
    if filename.endswith(".mp4"):
        video_file = Path(os.path.join(directory, filename))
        print(video_file)

        # video_file = Path("/gdrive/My Drive/bvh/1080/p9.mp4") 
        output_dir = Path(f'miscs/{video_file.stem}_cache')

        !rm -rf $output_dir

        if not output_dir.exists():
            os.makedirs(output_dir)
            
        cap = cv2.VideoCapture(str(video_file))
        totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        bar = progressbar.ProgressBar(max_value=totalFrames)
        keypoints_list = []
        img_width, img_height = None, None
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            nextFrameNo = cap.get(cv2.CAP_PROP_POS_FRAMES)
            bar.update(nextFrameNo)

            img_height = frame.shape[0]
            img_width = frame.shape[1]
            
            # returned shape will be (num_of_human, 25, 3)
            # last dimension includes (x, y, confidence)
            keypoints = e2d.estimate(img_list=[frame])[0]
            if isinstance(keypoints, np.ndarray) and len(keypoints.shape) == 3:
                keypoints_list.append(keypoints[0])

        cap.release()
        print("")

        # filter out failed result
        keypoints_list = smooth.filter_missing_value(
            keypoints_list=keypoints_list,
            method='ignore' # interpolation method will be implemented later
        )

        # smooth process will be implemented later

        # save 2d pose result
        pose2d = np.stack(keypoints_list)[:, :, :2]
        pose2d_file = Path(output_dir / '2d_pose.npy')
        np.save(pose2d_file, pose2d)
        print("Saved to:")
        print(pose2d_file)

        # =====# Initialize 3D pose estimator
        print("Initialize 3D pose estimator ...")
        importlib.reload(estimator_3d)
        e3d = estimator_3d.Estimator3D(
            config_file='models/openpose_video_pose_243f/video_pose.yaml',
            checkpoint_file='models/openpose_video_pose_243f/best_58.58.pth'
        )

        # =====
        # Estimate 3D pose from 2D pose
        print("Estimate 3D pose from 2D pose ...")
        # import torch
        # torch.cuda.empty_cache()
        pose2d = np.load(pose2d_file)
        pose3d = e3d.estimate(pose2d, image_width=img_width, image_height=img_height)

        # ======
        # convert coordinates
        print("convert coordinates ...")
        subject = 'S1'
        cam_id = '55011271'
        cam_params = camera.load_camera_params('cameras.h5')[subject][cam_id]
        R = cam_params['R']
        T = 0
        azimuth = cam_params['azimuth']

        pose3d_world = camera.camera2world(pose=pose3d, R=R, T=T)
        pose3d_world[:, :, 2] -= np.min(pose3d_world[:, :, 2]) # rebase the height

        pose3d_file = output_dir / '3d_pose.npy'
        np.save(pose3d_file, pose3d_world)

        # =====
        # Create bvh
        print("Create bvh ...")
        bvh_file = output_dir / f'{video_file.stem}.bvh'
        cmu_skel = cmu_skeleton.CMUSkeleton()
        channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)

        output = f'miscs/h36m_{video_file.stem}.bvh'
        h36m_skel = h36m_skeleton.H36mSkeleton()
        _ = h36m_skel.poses2bvh(pose3d_world, output_file=output)

        dist_full_path = f'"{directory}"'
        !cp {output} {dist_full_path}
        print(f"COPIED: {dist_full_path}")
        #perform dowmload
        # from google.colab import files
        # files.download(output)
    else:
        continue
print("DONE...")